# CS 520 Final Project

## Packages & Data Structures

In [1]:
using MatrixDepot, SparseArrays, AMD, LinearAlgebra, Statistics, Printf, JuMP, GLPK

[ Info: verify download of index files...
[ Info: reading database
[ Info: adding metadata...
[ Info: adding svd data...
[ Info: writing database
[ Info: used remote sites are sparse.tamu.edu with MAT index and math.nist.gov with HTML index


In [2]:
mutable struct IplpSolution
  x::Vector{Float64} # the solution vector 
  flag::Bool         # a true/false flag indicating convergence or not
  cs::Vector{Float64} # the objective vector in standard form
  As::SparseMatrixCSC{Float64} # the constraint matrix in standard form
  bs::Vector{Float64} # the right hand side (b) in standard form
  xs::Vector{Float64} # the solution in standard form
  lam::Vector{Float64} # the solution lambda in standard form
  s::Vector{Float64} # the solution s in standard form
end

mutable struct IplpProblem
  c::Vector{Float64}
  A::SparseMatrixCSC{Float64} 
  b::Vector{Float64}
  lo::Vector{Float64}
  hi::Vector{Float64}
end

function convert_matrixdepot(P::MatrixDepot.MatrixDescriptor)
  # key_base = sort(collect(keys(mmmeta)))[1]
  return IplpProblem(
    vec(P.c), P.A, vec(P.b), vec(P.lo), vec(P.hi))
end

convert_matrixdepot (generic function with 1 method)

## Presolve

In [3]:
function remove_zero_columns(problem::IplpProblem)
    zero_columns = findall(sum(problem.A .!= 0, dims=1)[:] .== 0)

    # Deduce solution values x_i
    x_values = zeros(length(problem.c))
    for col in zero_columns
        if problem.c[col] < 0
            if problem.hi[col] == Inf
                return :unbounded
            else
                x_values[col] = problem.hi[col]
            end
        elseif problem.c[col] > 0
            x_values[col] = problem.lo[col]
        end
    end

    # Remove zero columns from A
    problem.A = problem.A[:, setdiff(1:size(problem.A, 2), zero_columns)]

    # Adjust the objective function coefficients and bounds
    problem.c = problem.c[setdiff(1:length(problem.c), zero_columns)]
    problem.lo = problem.lo[setdiff(1:length(problem.lo), zero_columns)]
    problem.hi = problem.hi[setdiff(1:length(problem.hi), zero_columns)]

    # Adjust the constraint vector b
    problem.b -= problem.A * x_values[setdiff(1:length(x_values), zero_columns)]

    return problem
end

function remove_zero_rows(problem::IplpProblem)
    zero_rows = findall(sum(problem.A .!= 0, dims=2)[:] .== 0)
    non_zero_rows = setdiff(1:size(problem.A, 1), zero_rows)
    problem.A = problem.A[non_zero_rows, :]
    problem.b = problem.b[non_zero_rows]
    return problem
end

remove_zero_rows (generic function with 1 method)

## Standard Form

In [4]:
function convert_to_standard_form(P::IplpProblem)
    inf = 1.0e300
    m, n0 = size(P.A)

    # Initialize new matrices and vectors
    A_new = spzeros(m, 0)
    c_new = Vector{Float64}()

    for i in 1:n0
        lo_inf = P.lo[i] < -inf
        hi_inf = P.hi[i] > inf

        if lo_inf && hi_inf
            # x_i is unbounded: introduce x_i_pos and x_i_neg
            A_new = [A_new P.A[:, i] -P.A[:, i]]
            append!(c_new, [P.c[i], -P.c[i]])
        elseif lo_inf
            # x_i has an upper bound only: negate x_i
            A_new = [A_new -P.A[:, i]]
            append!(c_new, -P.c[i])
        else
            # x_i has a lower bound (or both): shift x_i
            A_new = [A_new P.A[:, i]]
            append!(c_new, P.c[i])
            P.b -= P.A[:, i] * P.lo[i]
        end
    end

    # Add slack variables for bounded variables
    idx_bounded = findall(x -> x > -inf && x < inf, P.hi)
    A_new = [A_new spzeros(m, length(idx_bounded))]
    c_new = vcat(c_new, zeros(length(idx_bounded)))
    I_bounded = Matrix{Float64}(I, length(idx_bounded), length(idx_bounded))

    As = [A_new; spzeros(length(idx_bounded), n0) I_bounded]
    bs = vcat(P.b, P.hi[idx_bounded] - P.lo[idx_bounded])
    cs = c_new

    lo_new = zeros(size(As, 2))
    hi_new = ones(size(As, 2)) * inf
    standard_form_ip = IplpProblem(cs, As, bs, lo_new, hi_new)

    return standard_form_ip
end

convert_to_standard_form (generic function with 1 method)

## Cholesky Factorization

In [5]:
function select_modified_cholesky_parameters(A::SparseMatrixCSC{Float64})
    beta = sqrt(cond(Matrix(A)))
    delta = eps(Float64) * norm(Matrix(A))
    return delta, beta
end

function modified_cholesky(A::SparseMatrixCSC{Float64}, delta::Real, beta::Real)
    @assert (n=size(A, 1)) == size(A, 2) "Input matrix must be a square matrix!"
    
    reorder = amd(A)
    L = A[reorder,reorder]

    d = ones(n)
    D = Diagonal(d)
        
    for i = 1:n-1
        theta = maximum(abs.(L[i+1:n, i]))
        d[i] = max(abs(L[i, i]), (theta / beta)^2, delta)
        L[i:n, i] ./= d[i]
        L[i, i] = 1.0
        L[i+1:n, i+1:n] .-= d[i] * (L[i+1:n, i] * L[i+1:n, i]')
    end

    d[n] = max(abs(L[n, n]), delta)
    L[n, n] = 1.0

    return (L=LowerTriangular(L), D=D, M=L, O=reorder)
end

function get_cholesky_lowtriangle(matrix, default=true)
    if default
        return cholesky(matrix)
    else
        delta, beta = select_modified_cholesky_parameters(matrix)
        F = modified_cholesky(matrix, delta, beta)
        return F.L
    end
end

get_cholesky_lowtriangle (generic function with 2 methods)

## Starting Point

In [6]:
function starting_point(P::IplpProblem, default_cholesky=true)
    # 4 Steps
    # (1) Solving Primal and Dual starting points least squares
    # (2) Calculate delta primal and delta dual
    # (3) Calculate delta primal hat and delta dual hat
    # (4) Get starting point
    
    symm = P.A*P.A'
    if default_cholesky
        f = get_cholesky_lowtriangle(symm)
    else
        f = get_cholesky_lowtriangle(symm, false)
    end
    
    # (1)
    # Solve Primal
    d = f\P.b
    x_hat = P.A'*d
    
    # Solve Dual
    λ_hat = f\(P.A*P.c)
    s_hat = P.c - P.A'*λ_hat
    
    # (2)
    delta_p = max(-1.5*minimum(x_hat), 0)
    delta_d = max(-1.5*minimum(s_hat), 0)
    
    n = size(x_hat,1)
    e = ones(n,1)
    
    # (3)
    x_hat = x_hat + delta_p*e
    s_hat = s_hat + delta_d*e

    numerator = 0.5 * (x_hat'*s_hat)
    delta_p_hat = (numerator / (e'*s_hat))[1]
    delta_d_hat = (numerator / (e'*x_hat))[1]

    # (4)
    x_0 = x_hat + delta_p_hat*e
    λ_0 = λ_hat
    s_0 = s_hat + delta_d_hat*e

    return (x=x_0, λ=λ_0, s=s_0, f=f)
end

starting_point (generic function with 2 methods)

## Predictor-Corrector

In [7]:
function build_kkt_matrix(A, x, s)
    m, n = size(A)
    zero_mn = spzeros(m, n)
    zero_nm = spzeros(n, m)
    zero_mm = spzeros(m, m)
    zero_nn = spzeros(n, n)

    X = spdiagm(0 => x[:, 1])
    Identity = Matrix{Float64}(I,n,n)
    S = spdiagm(0 => s[:, 1])

    M = [zero_mm  A         zero_mn;
         A'       zero_nn   Identity;
         zero_nm  S         X]

    return M
end

function solve_linear_system(A,x,s,rb,rc,rxs)
    KKT = build_kkt_matrix(A, x, s)
    f = lu(KKT)
    m = length(rb)
    n = length(rc)
    b = Array{Float64}([-rb; -rc; -rxs])
    b = f\b
    dlam = b[1:m]
    dx = b[1+m:m+n]
    ds = b[1+m+n:m+2*n]
    return dlam,dx,ds
end

function alpha_max(x,dx,hi=1.0)
    alpha = hi
    for i = 1:length(x)
        alpha = dx[i] < 0 ? min(alpha, -x[i] / dx[i]) : alpha
    end
    if alpha < 0
        alpha = Inf
    end
    alpha = min(alpha,hi)
    return alpha
end

function unbound_check(alpha_p, alpha_d)
    if alpha_p > 1e308 || alpha_d > 1e308
       return true 
    end
    return false
end

function select_gamma(P::IplpProblem)
    m, n = size(P.A)
    problem_size = max(m, n)
    
    if problem_size <= 100
        gamma = 0.01
    elseif problem_size <= 1000
        gamma = 0.05
    else
        gamma = 0.1
    end
    
    return gamma
end

select_gamma (generic function with 1 method)

In [8]:
function predictor_corrector(P::IplpProblem, info=true, tol=1e-8, maxit=100)    
    iter = 0
    SP = starting_point(P)
    GAMMA = select_gamma(P) #limit step length
    m,n = size(P.A)
    
    x_0 = copy(SP.x)
    λ_0 = copy(SP.λ)
    s_0 = copy(SP.s)
    
    if info
        @printf("%10s %12s %12s\n", "Iteration", "Alpha Prim", "Alpha Dual")
        @printf("%10d %12.4g %12.4g\n", iter, 0., 0.)
    end
    
    x1,λ1,s1,flag = vec([0.]),vec([0.]),vec([0.]),false
    
    for iter=1:maxit
        # Residuals for primal feasibility       rb
        #               dual feasibility         rc
        #               complementary slackness  rxs
        rb  = P.A*x_0-P.b
        rc  = P.A'*λ_0+s_0-P.c
        rxs = x_0.*s_0

        # Predictor
        λ_aff,x_aff,s_aff = solve_linear_system(P.A,x_0,s_0,rb,rc,rxs) # direction
        alpha_aff_primal  = alpha_max(x_0,x_aff) # step lengths
        alpha_aff_dual = alpha_max(s_0,s_aff)
        
        # Corrector
        μ = mean(rxs)
        μ_affine = sum((x_0 .+ alpha_aff_primal .* x_aff) .* (s_0 .+ alpha_aff_dual .* s_aff)) / n
        σ = (μ_affine/μ)^3 # centering parameter
        rxs = x_aff.*s_aff.-σ*μ
        λ_cc,x_cc,s_cc = solve_linear_system(P.A,x_0,s_0,spzeros(m),spzeros(n),rxs)

        # Combine to find search direction
        direction_x = x_aff+x_cc
        direction_λ = λ_aff+λ_cc
        direction_s = s_aff+s_cc
        
        # Finding the max step length it can take 
        # Find max allowable step len Wright Chapter 4
        alpha_max_pri, alpha_max_dual = alpha_max(x_0, direction_x, Inf), alpha_max(s_0, direction_s, Inf)
        
        # Wright P205 Chapter 10
        x1_pri, s1_dual = x_0 + alpha_max_pri * direction_x, s_0 + alpha_max_dual * direction_s
        μp= dot(x1_pri, s1_dual) / n
        f_prim = (GAMMA * μp / s1_dual[argmin(x1_pri)] - x_0[argmin(x1_pri)]) / alpha_max_pri / direction_x[argmin(x1_pri)]
        f_dual = (GAMMA * μp / x1_pri[argmin(s1_dual)] - s_0[argmin(s1_dual)]) / alpha_max_dual / direction_s[argmin(s1_dual)]
        alpha_pri, alpha_dual = max(1 - GAMMA, f_prim) * alpha_max_pri, max(1 - GAMMA, f_dual) * alpha_max_dual

        
        # Unboundness check
        if unbound_check(alpha_pri, alpha_dual)
            warn("This problem is unbounded")
            return x1,λ1,s1,false,iter
        end
        
        # Take the step
        x1 = x_0+alpha_pri*direction_x
        λ1 = λ_0+alpha_dual*direction_λ
        s1 = s_0+alpha_dual*direction_s
        x_0 = x1
        λ_0 = λ1
        s_0 = s1
        
        if info
            @printf("%10d %12.4g %12.4g\n", iter, alpha_pri, alpha_dual)
        end
        
        r1 = norm(P.A*x1-P.b)/(1+norm(P.b))
        r2 = norm(P.A'*λ1+s1-P.c)/(1+norm(P.c))
        r3 = abs(dot(P.c,x1)-dot(P.b,λ1))/(1+abs(dot(P.c,x1)))
        if r1 < tol && r2 < tol && r3 < tol
            flag = true
            break
        end
    end
    
    return x1,λ1,s1,flag
end

predictor_corrector (generic function with 4 methods)

In [9]:
function iplp(P::IplpProblem, info=true, tol=1e-8, maxit=100)
    P = remove_zero_columns(P)
    P = remove_zero_rows(P)
    stdpb = convert_to_standard_form(P)
    
    # Check Infeasibility
    @printf("===================Infeasible Check=====================\n")
    m,n = size(stdpb.A)
    A = [stdpb.A Matrix{Float64}(I,m,m)]
    b = copy(stdpb.b)
    c = [zeros(Float64, n); ones(Float64, m)]
    check_target = IplpProblem(c, A, b, vec(P.lo), vec(P.hi))
    check_x = predictor_corrector(check_target, false)[1]
    if abs(dot(c, check_x)) > 1e-8
        @printf("Infeasible LP Problem!\n")
        return IplpSolution(vec([0.]),false,vec(c),A,vec(b),vec([0.]),vec([0.]),vec([0.])), -1
    else
        @printf("Feasible LP Problem.\n")
    end
        
    if info
        @printf("\n=======================Solution=========================\n")
        @printf("The problem is feasible, finding solution now...\n")
        x_sol,λ_sol,s_sol,flag = @time predictor_corrector(stdpb)
    else
        x_sol,λ_sol,s_sol,flag = @time predictor_corrector(stdpb,false)
    end
        
    op_val = dot(stdpb.c, x_sol)
    if info
        @printf("Optimal Value: %.5f.\n", op_val)
    end
        
    return IplpSolution(vec(x_sol),flag,vec(stdpb.c),stdpb.A,vec(stdpb.b),vec(x_sol),vec(λ_sol),vec(s_sol)), op_val
end

iplp (generic function with 4 methods)

In [10]:
function get_correct_ov(problem_name)
    if occursin("lpi", problem_name)
       return Inf
    end
    md_info = split(Markdown.plain(mdinfo("LPnetlib/" * problem_name)), "\n")
    header_line = "Name       Rows   Cols   Nonzeros    Bytes  BR      Optimal Value"
    header_idx = -1
    summary = ""
    for i in 1:length(md_info)
        if occursin(header_line, md_info[i])
            header_idx = i
            break
        end
    end
    
    summary = strip(md_info[header_idx+1], ['*',  ' '])
    pattern = r"(\S+)\s+(\d+)\s+(\d+)\s+(\d+)\s+(\d+)\s+(\S{0,1})\s+(-?\d+\.\d+E[+-]\d+)"
    m = match(pattern, summary)
    if m !== nothing
        parsed_dict = Dict(
            "Name" => m.captures[1],
            "Rows" => parse(Int, m.captures[2]),
            "Cols" => parse(Int, m.captures[3]),
            "Nonzeros" => parse(Int, m.captures[4]),
            "Bytes" => parse(Int, m.captures[5]),
            "BR" => m.captures[6],
            "OV" => parse(Float64, m.captures[7])
        )
    else
        println("No match found")
        parsed_dict = nothing
    end

    return parsed_dict["OV"]   
end

function test(problem_name::String, info=true)
    md = mdopen("LPnetlib/" * problem_name)
    pb = convert_matrixdepot(md)
    correct_ov = get_correct_ov(problem_name)

    solution, op_val = iplp(pb, info)
    
    if solution.flag
        @printf("Solution found.\n")
    else
        @printf("Not solution found after max iteration.\n")
    end
    
    @printf("\n======================Comparison========================\n")
    
    if (correct_ov != Inf && !solution.flag) || (correct_ov == Inf && solution.flag)
        println("Not matched with correct result")
    else
        if correct_ov == Inf
            println("Successfully detect infeasibility")
        else
            println("Checking difference...")
            diff = correct_ov-op_val
            absolute_diff = abs(diff)
            relative_diff = abs(diff/correct_ov)
            if info
                println("Absolute difference: ", absolute_diff)
                println("Relative difference: ", relative_diff)
            end
            
            if relative_diff < 1e-3
                println("Result is consistent with correct optimal value")
            else
                println("Result is wrong")
            end
        end
        
    end
    
    return (cov=correct_ov, op=op_val)
end

function prompt_test(info=true)
    @printf("Which test problem would you like to solve? \n(Input as: lp_afiro, etc)\n")
    problem_name = readline()
    test(problem_name, info)
end

prompt_test (generic function with 2 methods)

In [62]:
prompt_test()

Which test problem would you like to solve? 
(Input as: lp_afiro, etc)
stdin> lp_brandy
===================Infeasible Check=====================
Feasible LP Problem.

=======================Solution=========================
The problem is feasible, finding solution now...
 Iteration   Alpha Prim   Alpha Dual
         0            0            0
         1       0.7837       0.9183
         2       0.3681       0.3039
         3       0.3645       0.2844
         4       0.2867       0.6589
         5        0.185       0.3998
         6      0.04863      0.07348
         7      0.01858      0.01295
         8       0.2076       0.1869
         9       0.4545       0.5522
        10       0.8853       0.8895
        11       0.9154       0.8306
        12       0.4653       0.8962
        13       0.5664       0.7958
        14       0.8177       0.6822
        15       0.8386       0.7924
        16       0.9198       0.4358
        17        0.802       0.9615
        18       0.9896 

(abs = 1518.5098964881429, rel = Inf)

In [65]:
problems = ["lp_afiro","lp_brandy","lp_fit1d","lp_adlittle",
            "lp_agg","lp_ganges","lp_stocfor1", "lp_25fv47", "lpi_chemcom"];

In [88]:
for problem_name in problems
    try
        println("-> Testing: ", problem_name)
        D = test(problem_name, false)
        println(D.op)
    catch e
        println("An error occurred: ", e)
    end
    println("--------------------")
end

-> Testing: lp_afiro
===================Infeasible Check=====================
Feasible LP Problem.
  0.003258 seconds (3.88 k allocations: 3.610 MiB)
Solution found.
-464.7531428571428
--------------------
-> Testing: lp_brandy
===================Infeasible Check=====================
Feasible LP Problem.
  0.101283 seconds (11.77 k allocations: 125.460 MiB, 4.38% gc time)
Solution found.
1518.5098964881429
--------------------
-> Testing: lp_fit1d
===================Infeasible Check=====================
Feasible LP Problem.
An error occurred: SingularException(0)
--------------------
-> Testing: lp_adlittle
===================Infeasible Check=====================
Feasible LP Problem.
  0.014020 seconds (5.53 k allocations: 16.288 MiB)
Solution found.
225494.9631824746
--------------------
-> Testing: lp_agg
===================Infeasible Check=====================
Feasible LP Problem.
  0.259092 seconds (22.20 k allocations: 418.168 MiB, 2.63% gc time)
Solution found.
-3.599176728657631

In [11]:
test("lp_ganges")

===================Infeasible Check=====================
Feasible LP Problem.

=======================Solution=========================
The problem is feasible, finding solution now...
 Iteration   Alpha Prim   Alpha Dual
         0            0            0
         1       0.5435       0.9281
         2       0.8042        0.873
         3       0.7363       0.6958
         4       0.8353       0.5409
         5        0.963       0.9009
         6       0.6181        0.564
         7       0.4952       0.5332
         8       0.8552       0.5716
         9          0.9       0.8094
        10         0.58       0.8694
        11       0.8365       0.7639
        12       0.8227       0.9409
        13       0.8776       0.9609
        14       0.7584       0.4199
        15       0.6685       0.5832
        16       0.6421       0.6214
        17       0.6208       0.7385
        18       0.7312       0.8761
        19       0.8104       0.8101
        20       0.8143       0.9787
 

(abs = -1, rel = -1, op = -247333.29318261007)